In [12]:
from requests_html import HTMLSession
import json


In [13]:
with open("urls1.txt", "r") as f:
    string = f.read()
    # stringList = string.split("https")
    # stringList.pop(0)
    # stringFnl = list(map(lambda x: "https"+x, stringList))


In [16]:
stringFnl = string.split('\n')
stringFnl.pop()
len(stringFnl)


10000

In [17]:
stringFnl[0]



'https://hk.centanet.com/findproperty/zh-cn/transaction-detail/-%E4%BC%9F%E5%8D%8E%E4%B8%AD%E5%BF%83_FTY202203S0762'

In [18]:
stringFnl[-1]

'https://hk.centanet.com/findproperty/zh-cn/transaction-detail/-%E8%8D%94%E5%A1%98%E5%A4%A7%E5%8E%A6_20072000950020'

In [35]:
def collect_info(url):
    r = session.get(url, headers = header)
    info = dict()
    info["rooms"] = None
    info["age"] = None
    info["direction"] = None
    info["priceNum"] = None
    info["priceLbl"] = None
    info["area"] = None
    info["location"] = None
    info["gfa"] = None
    info["cfa"] = None
    
    #price
    try:
        priceNum = r.html.xpath("//span[@class='price-lbl']/text()")[0].strip()
        priceLbl = r.html.xpath(
            "//span[@class='price-lbl']/label/text()")[0].strip()
        info["priceNum"] = priceNum
        info["priceLbl"] = priceLbl
    except IndexError:
        pass

    #location and area
    try:
        area = r.html.xpath(
            "//span[@class='el-breadcrumb__item'][5]/span/a/text()")[0].strip()
        info["area"] = area
    except IndexError:
        pass
    
    try:
        location = r.html.xpath(
            "//h1[@class='info-title f-middle']/p/text()")[0].strip()
        info["location"] = location
    except IndexError:
        pass
    
    #gfa cfa
    try:
        gfa = r.html.xpath("//div[@class='fl']/p/span[2]/text()")[0].strip()
        info["gfa"] = gfa
    
    except IndexError:
        pass
    try:
        cfa = r.html.xpath("//div[@class='fr']/p/text()")[0].strip()
        info["cfa"] = cfa
    except IndexError:
        pass
    #rooms age direction
    
    for divID in range(len(r.html.xpath("// div[@class='info-num flex-between']/div"))):
        div = divID+1
        cat = r.html.xpath(
            "// div[@class='info-num flex-between']/div[{}]/p[1]/text()".format(div))[0].strip()
        content = r.html.xpath(
            "// div[@class='info-num flex-between']/div[{}]/p[2]/text()".format(div))[0].strip()
        if cat == "楼龄":
            info["age"] = content
        elif cat == "座向":
            info["direction"] = content
        elif cat == "间隔":
            info["rooms"] = content

    return info


In [30]:
session = HTMLSession()

header = {

    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'
}


In [42]:
import time
data_lst = []
fail_lst = []
for i in stringFnl:
    d = collect_info(i)
    time.sleep(5)
    if d['location'] == None:
        count = 0
        print("retrying:")
        print(i)
        while count <= 10 and d['location'] == None:
            count += 1
            time.sleep(5+count)
            d = collect_info(i)
        if d['location'] == None:
            print('fail')
            fail_lst.append(i)
            continue
        else:
            print('success')
    data_lst.append(d)
    count = 0


ChunkedEncodingError: ("Connection broken: ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)", ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))

In [43]:
len(data_lst)

234

In [ ]:
import pandas as pd  
pd.DataFrame(data_lst).to_csv('data1.csv', index = False)

In [43]:
cnt = 0
with open('data.json',mode='w', encoding='utf-8') as f:
    for i in stringFnl:
        cnt+=1
        #print(cnt,"working with:",i)
        print("*",end="")
        info = collect_info(i)
        json.dump(info,f)
        #print("done")

****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

In [41]:
collect_info("https://hk.centanet.com/findproperty/zh-cn/transaction-detail/-%E6%B5%B7%E8%81%94%E5%B9%BF%E5%9C%BA_22030401730038")


{'rooms': None,
 'age': '27 年',
 'direction': None,
 'priceNum': '495',
 'priceLbl': '万',
 'area': '粉岭',
 'location': '海联广场 2座 5楼 G室',
 'gfa': '442',
 'cfa': '584'}

In [30]:
len("***************")


15

In [62]:
with open("data.json","r") as f:
    s = f.readlines()[0]

In [63]:
len(s)

1433015

In [67]:
lst = s.split("{")

In [71]:
lst[1]

'"rooms": "2\\u623f", "age": "17 \\u5e74", "direction": "\\u4e1c\\u5317", "priceNum": "898", "priceLbl": "\\u4e07", "area": "\\u5751\\u53e3", "location": "\\u851a\\u84dd\\u6e7e\\u7554 5\\u5ea7 \\u4e2d\\u5c42 G\\u5ba4", "gfa": "490", "cfa": "657"}'

In [72]:
lst = list(map(lambda x:"{"+x, lst))

In [79]:
lst1 = list(map(lambda x :json.loads(x), lst))

In [80]:
with open("data1.json","w") as f:
    json.dump(lst1,f)


In [85]:
f.close()

In [44]:
import pandas as pd

In [83]:
df = pd.DataFrame(lst1)

In [84]:
df.iloc[:10]

,rooms,age,direction,priceNum,priceLbl,area,location,gfa,cfa
0,2房,21 年,北,978.8,万,坚尼地城,加惠台 1座 低层 C室,464,589
1,2房,17 年,东北,898,万,坑口,蔚蓝湾畔 5座 中层 G室,490,657
2,None,49 年,None,494,万,湾仔,永德大厦 B座 4楼 B8室,373,500
3,None,33 年,东,455,万,元朗市中心,荣丰大厦 6楼 B室,423,616
4,None,27 年,西,"1,000",万,马鞍山,雅典居 7座 17楼 D室,727,901
5,2房,8 年,西北,618,万,元朗东南,尚悦 1座 19楼 D室,425,576
6,None,27 年,西,"1,038",万,马鞍山,雅典居 9座 22楼 D室,727,901
7,None,31 年,None,662,万,粉岭,粉岭中心 1期 G座 中层 3室,535,694
8,None,57 年,南,"1,360",万,何文田,麒麟阁 低层 E室(244号),"1,075","1,293"
9,2房,22 年,东北,718,万,坑口,新宝城 1座 中层 G室,377,527
